In [2]:
from dotenv import load_dotenv
import os
import pandas as pd
import matplotlib.pyplot as plt

from eth_rpc import PrivateKeyWallet
from emp_orderly.utils import from_address
from emp_orderly import (
    Strategy, EmpOrderly,
    crossover, plot_heatmaps,
    EMA, SMA, SLOPE, CHOP,
    EmpyrealOrderlySDK,
)
from emp_orderly.onboarding.faucet import request_testnet_funds as faucet
from emp_orderly.onboarding import deposit
from emp_orderly_types import PerpetualAssetType, Interval, OrderType

load_dotenv()

True

In [3]:
wallet = PrivateKeyWallet(private_key=os.getenv("PRIVATE_KEY"))
orderly_id = from_address(wallet.address)

sdk = EmpyrealOrderlySDK(pvt_hex=wallet.private_key, account_id=orderly_id, is_testnet=True)

In [4]:
print(wallet.address)

0xd3E45819C43e582C7Ecfe2e7BFcD2544a7e53e4c


In [5]:
await faucet(wallet)

False

In [6]:
await deposit(wallet)

'0xac8396aaa238c19212162451bd5c9a3779693fe9e6591844cff493347547b9bb'

In [7]:
print(await wallet.balance())

109545375600000000


In [8]:
class EMSB(Strategy):
    """EMA Momentum and Slope Breakout Strategy.
    
    order_size: The amount of order that can be 
        made at a time.
    days_length_1: 12 days length for 
        calculating 12 days EMA.
    days_length_2: 26 days length for 
        calculating 26 days EMA.
    length: The number of days the Strategy is 
        intended to run for.
    """

    order_size: float = 0.5
    days_length_1: int = 12
    days_length_2: int = 26
    length: int = 14
    
    @classmethod
    def update_lags(cls, days_length_1, days_length_2, length):
        cls.days_length_1 = days_length_1
        cls.days_length_2 = days_length_2
        cls.length = length

    def init(self):
        close = self.data.close
        self.sma_days_1 = self.I(EMA, close, self.days_length_1)
        self.sma_days_2 = self.I(EMA, close, self.days_length_2)

    def next(self):
        if (
            crossover(self.sma_days_1, self.sma_days_2)
        ):
            self.position.close()
            self.buy(size=self.order_size)
        elif (
            crossover(self.sma_days_2, self.sma_days_1)
        ):
            self.position.close()
            self.sell(size=self.order_size)


tester = EmpOrderly(
    cash=1000,
    commission=.0001,
    exclusive_orders=True,
    sdk=sdk,
)

# load strategy and data
tester.set_strategy(EMSB)
await tester.load_data(
    lookback=14,
    interval=Interval.five_minute,
    asset=PerpetualAssetType.APT,
)
# print(tester._history[tester._history["volume"] > 0])

# backtest
tester.backtest()

Start                     2024-09-17 14:00:00
End                       2024-09-21 09:15:00
Duration                      3 days 19:15:00
Exposure Time [%]                   97.080292
Equity Final [$]                  1096.666956
Equity Peak [$]                   1109.763157
Return [%]                           9.666696
Buy & Hold Return [%]               21.128922
Return (Ann.) [%]                 13147.12813
Volatility (Ann.) [%]             3984.786893
Sharpe Ratio                          3.29933
Sortino Ratio                             inf
Calmar Ratio                     10015.200827
Max. Drawdown [%]                   -1.312717
Avg. Drawdown [%]                   -0.415796
Max. Drawdown Duration        1 days 01:10:00
Avg. Drawdown Duration        0 days 04:47:00
# Trades                                   20
Win Rate [%]                             40.0
Best Trade [%]                       9.450381
Worst Trade [%]                     -0.884155
Avg. Trade [%]                    

In [9]:
# # plot
# tester.plot(show_price_data=False)
# plt.show()